# Comparing alternative vertex embeddings to omni

## Vertex Weighting - doesn't work

Represent a vertex as the vector of edge weight connections to other vertices (i.e., a row of the adjacency matrix).
Compare vectors across strains via a Distance Correlation (this is effectively multivariate distance matrix regression, MDMR).

In [1]:
import numpy as np
import pandas as pd
from graspologic.datasets import load_mice
from hyppo.ksample import KSample
from statsmodels.stats.multitest import multipletests

In [2]:
# Load the full mouse dataset
mice = load_mice()

# Stack all adjacency matrices in a 3D numpy array
graphs = np.array(mice.graphs)

# Sort the connectomes and genotype labels so BTBR is first
label_indices = np.argsort(mice.labels).reshape(4, 8)
label_indices = label_indices[[1, 0, 2, 3]].reshape(-1)
labels = mice.labels[label_indices]
graphs = graphs[label_indices]

# Get sample parameters
n_subjects = mice.meta["n_subjects"]
n_vertices = mice.meta["n_vertices"]

In [3]:
# Split the set of graphs by genotype
btbr = graphs[labels == "BTBR"]
b6 = graphs[labels == "B6"]
cast = graphs[labels == "CAST"]
dba2 = graphs[labels == "DBA2"]

connectomes = [btbr, b6, cast, dba2]

In [4]:
vertex_pvals = []

for roi in range(332):

    # Get the (i,j)-th edge for each connectome
    samples = [genotype[:, roi, :] for genotype in connectomes]

    # Calculate the p-value for the (i,j)-th edge
    try:
        statistic, pvalue = KSample("Dcorr").test(*samples, reps=10000000)
    except ValueError:
        # A ValueError is thrown when any of the samples have equal edge
        # weights (i.e. one of the inputs has 0 variance)
        statistic = np.nan
        pvalue = 1

    vertex_pvals.append([roi, statistic, pvalue])

In [5]:
# Convert the nested list to a dataframe
signal_vertices = pd.DataFrame(vertex_pvals, columns=["ROI", "stat", "pvalue"])
signal_vertices.sort_values(by="pvalue", inplace=True, ignore_index=True)

# Correct p-values
reject, holm_pvalue, _, _ = multipletests(
    signal_vertices["pvalue"], alpha=0.05, method="holm"
)
signal_vertices["holm_pvalue"] = holm_pvalue
signal_vertices["significant"] = reject
signal_vertices.sort_values(by="holm_pvalue", inplace=True, ignore_index=True)
signal_vertices.head()

,ROI,stat,pvalue,holm_pvalue,significant
0,82,0.855475,9.993691e-08,0.000033,True
1,309,0.851650,1.064597e-07,0.000035,True
2,126,0.850944,1.077108e-07,0.000036,True
3,248,0.847117,1.147466e-07,0.000038,True
4,269,0.840477,1.280620e-07,0.000042,True


In [6]:
def lookup_roi_name(roi):
    roi -= 1
    hemisphere = "R" if roi // 166 else "L"
    roi = roi % 166
    structure = mice.atlas["Structure"].values[roi]
    structure = " ".join(structure.split("_"))
    return f"{structure} ({hemisphere})"

In [7]:
# Get the top 20 strongest signal edges
strong_signal_vertices = signal_vertices.head(20)
strong_signal_vertices["ROI"] = strong_signal_vertices["ROI"].apply(lookup_roi_name)
strong_signal_vertices

/var/folders/8l/r82lslb91qsd3qybmx787l180000gn/T/ipykernel_88811/298922132.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  strong_signal_vertices["ROI"] = strong_signal_vertices["ROI"].apply(lookup_roi_name)


,ROI,stat,pvalue,holm_pvalue,significant
0,Inferior Colliculus (L),0.855475,9.993691e-08,0.000033,True
1,Ventral Spinocerebellar Tract (R),0.851650,1.064597e-07,0.000035,True
2,Ventral Hippocampal Commissure (L),0.850944,1.077108e-07,0.000036,True
3,Inferior Colliculus (R),0.847117,1.147466e-07,0.000038,True
4,Trigeminal Motor Nucleus (R),0.840477,1.280620e-07,0.000042,True
5,Anterior Pretectal Nucleus (R),0.838293,1.327723e-07,0.000043,True
6,Trigeminal Motor Nucleus (L),0.838242,1.328856e-07,0.000043,True
7,Ectorhinal Cortex (L),0.838199,1.329790e-07,0.000043,True
8,Pararubral Nucleus (L),0.829333,1.539851e-07,0.000050,True
9,Fastigial Medial Dorsolateral Nucleus of Cereb...,0.824864,1.658030e-07,0.000054,True


In [8]:
signal_vertices.query("ROI == 120")

,ROI,stat,pvalue,holm_pvalue,significant
36,120,0.766701,4.345561e-07,0.000129,True


In [9]:
signal_vertices.query("ROI == 120 + 166")

,ROI,stat,pvalue,holm_pvalue,significant
45,286,0.759922,4.862772e-07,0.00014,True


## Graph Statistics - doesn't work

Use 5 common vertex-level graph statistics: degree, clustering, betweenness centrailty, closeness centrailty, and the number of triangles. 

In [10]:
from collections import defaultdict

import networkx as nx

In [13]:
def summarize(graph):

    # Define the network statistics to compute
    functions = [
        nx.degree,
        nx.clustering,
        nx.betweenness_centrality,
        nx.closeness_centrality,
        nx.triangles,
    ]

    # Get a dictionary of network statistics for each node in the graph
    G = nx.from_numpy_array(graph)
    stats = defaultdict(list)
    for func in functions:
        if func == nx.degree:
            for node, degree in G.degree():
                stats[node].append(degree)
        else:
            for node, scalar in func(G).items():
                stats[node].append(scalar)

    # Organize statistics in a pandas df
    df = pd.DataFrame.from_dict(stats, orient="index")
    df["node"] = df.index
    df.columns = [func.__name__ for func in functions] + ["node"]

    return df


# %% Embed all graphs
def embed(graphs):
    frames = [summarize(graph) for graph in graphs]
    df = pd.concat(frames)
    df.reset_index(level=0, inplace=True, drop=True)
    return df


In [14]:
graph_stats = []
for graph in connectomes:
    graph_stats.append(embed(graph))

In [15]:
vertex_pvals = []

for roi in range(332):

    # Get the (i,j)-th edge for each connectome
    samples = [Xhat.query(f"node == {roi}").drop(columns=["node"]).values for Xhat in graph_stats]

    # Calculate the p-value for the (i,j)-th edge
    try:
        statistic, pvalue = KSample("Dcorr").test(*samples, reps=10000000)
    except ValueError:
        # A ValueError is thrown when any of the samples have equal edge
        # weights (i.e. one of the inputs has 0 variance)
        statistic = np.nan
        pvalue = 1

    vertex_pvals.append([roi, statistic, pvalue])

In [16]:
# Convert the nested list to a dataframe
signal_vertices = pd.DataFrame(vertex_pvals, columns=["ROI", "stat", "pvalue"])
signal_vertices.sort_values(by="pvalue", inplace=True, ignore_index=True)

# Correct p-values
reject, holm_pvalue, _, _ = multipletests(
    signal_vertices["pvalue"], alpha=0.05, method="holm"
)
signal_vertices["holm_pvalue"] = holm_pvalue
signal_vertices["significant"] = reject
signal_vertices.sort_values(by="holm_pvalue", inplace=True, ignore_index=True)
signal_vertices.head()

,ROI,stat,pvalue,holm_pvalue,significant
0,35,0.534338,0.000021,0.006963,True
1,200,0.519373,0.000027,0.008929,True
2,34,0.502768,0.000036,0.011773,True
3,36,0.499924,0.000037,0.012314,True
4,23,0.496980,0.000039,0.012901,True


In [17]:
signal_vertices.query("ROI == 120")

,ROI,stat,pvalue,holm_pvalue,significant
93,120,0.254543,0.002493,0.595941,False


In [18]:
signal_vertices.query("ROI == 120 + 166")

,ROI,stat,pvalue,holm_pvalue,significant
97,286,0.252175,0.002599,0.610748,False
